I'm gonna overwrite a lot of this notebook's old content. I changed the way I'm calculating wt, and wanna test that my training worked. 

In [207]:
from pearce.emulator import OriginalRecipe, ExtraCrispy, SpicyBuffalo
from pearce.mocks import cat_dict
import numpy as np
from os import path

In [208]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [209]:
training_file = '/scratch/users/swmclau2/xi_zheng07_cosmo_lowmsat/PearceRedMagicXiCosmoFixedNd.hdf5'
#training_file = '/u/ki/swmclau2/des/PearceRedMagicXiCosmoFixedNdLowMsat.hdf5'
test_file = '/scratch/users/swmclau2/xi_zheng07_cosmo_test_lowmsat//PearceRedMagicXiCosmoFixedNd_Test.hdf5'
#test_file = '/u/ki/swmclau2/des/PearceRedMagicXiCosmoFixedNdLowMsatTest.hdf5'

em_method = 'gp'
split_method = 'random'

In [210]:
a = 1.0
z = 1.0/a - 1.0

In [211]:
fixed_params = {'z':z}#, 'cosmo': 0}#, 'r':24.06822623}

In [212]:
v = np.array([ 0.2661017,  0.1054246,  1.1295944,  0.3643993,  0.2408568,
    11.5649985,  5.6407612,  4.9071932, 10.6279446, 11.7621938,
     4.7031938,  6.3770235, 11.7578699, 11.7547548,  8.4866085,
    -12.0550382,  1.8339794, 10.6161248,  2.2441632, 13.8155106,
    10.6371797, 11.3512804,  7.342365 ,  3.1795786,  3.7658774,
     5.0188608,  4.6846614, 13.8155106, 13.8155106,  5.545777 ,
    13.8155106, -1.5383083, -13.8155106])

In [213]:
# guessing cosmo comes first? 
zx_kernel_pnames = ['omch2', 'ombh2', 'ln10As', 'H0', 'ns', 'Neff', 'w0', 'logM1', 'alpha', 'logM0', 'sigma_logM',None, None, None, None, 'amp']
zx_kernel_pnames.extend(zx_kernel_pnames)
zx_kernel_pnames.append('amp')

In [214]:
zx_metric = {}
for pname, val in zip(zx_kernel_pnames, v):
    if pname is None:
        continue
    if pname not in zx_metric:
        zx_metric[pname] = []
    zx_metric[pname].append(val)

In [215]:
param_names = ['ombh2', 'omch2', 'w0', 'ns', 'ln10As', 'H0', 'Neff', 'logM0', 'sigma_logM', 'logM1', 'alpha']

In [216]:
m1 = [zx_metric[pname][0] for pname in param_names]
m2 = [zx_metric[pname][0] for pname in param_names]
a1, a2, a3 = zx_metric['amp']

In [217]:
from george.kernels import *

In [ ]:
#kernel = ExpSquaredKernel(np.exp(m1), ndim=11) * ConstantKernel(a1, ndim=11)\
#+ Matern32Kernel(np.exp(m2), ndim=11) + ConstantKernel(a2, ndim=11)#+ WhiteKernel(a3, ndim=11)
kernel = Matern52Kernel(np.exp(0),ndim=11)+ ConstantKernel(0, ndim = 11)

In [ ]:
np.random.seed(0)
emu = SpicyBuffalo(training_file, method = em_method, hyperparams={'kernel':kernel}, fixed_params=fixed_params,
                 custom_mean_function = 'linear', downsample_factor = 0.1)

In [ ]:
for pname, val in zip(emu._emulators[0].get_parameter_names(), emu._emulators[0].get_parameter_vector()):
    print pname, val

In [ ]:
emu.scale_bin_centers

In [ ]:
rbins = np.logspace(-1.1, 1.6, 19)
print (rbins[1:]+rbins[:-1])/2

In [ ]:
params = {}
for pname in emu.get_param_names():
    if pname == 'r':
        continue
    low, high = emu.get_param_bounds(pname)
    params[pname] = np.random.uniform(low, high)
    
print params

In [ ]:
pred_y = emu.emulate_wrt_r(params)[0]
print pred_y

In [ ]:
plt.plot(emu.scale_bin_centers, pred_y)
plt.xscale('log')

In [ ]:
gof = emu.goodness_of_fit(test_file, statistic = 'frac')
print gof.mean(), np.median(gof)
#for g in gof:
#    print g.mean(), np.median(g)

In [205]:
#print g.mean(), np.median(g)

In [206]:
pred_y, y = emu.goodness_of_fit(test_file, statistic = None)
#print gof.mean(), np.median(gof)

KeyboardInterrupt: 

In [ ]:
plt.hist(y)

In [ ]:
plt.hist(pred_y)

In [ ]:
plt.hist(emu.y)

In [ ]:
plt.hist(np.log10(gof));

In [ ]:
plt.hist(emu.y)

In [ ]:
for i in xrange(50):    
    params = {}

    for pname in emu.get_param_names():
        if pname == 'r':
            continue
        low, high = emu.get_param_bounds(pname)
        params[pname] = np.random.uniform(low, high)
    pred_y = emu.emulate(params)[0]
    print pred_y
    #print params

In [ ]:
for i, (g, r) in enumerate(zip(gof, emu.scale_bin_centers)):
    print r, g.mean(), np.median(g)
    #plt.hist(np.log10(g))
    #plt.show()

In [ ]:
n_cosmo_params = 7
loo_cosmo = emu.x[0, 0,  :n_cosmo_params]

loo_cosmo_idxs = np.all(emu.x[:, :,:n_cosmo_params] == loo_cosmo, axis =2)
train_x, train_y, train_yerr = emu.x[~loo_cosmo_idxs, :], emu.y[ ~loo_cosmo_idxs], emu.yerr[ ~loo_cosmo_idxs]
test_x, test_y, test_yerr = emu.x[loo_cosmo_idxs, :], emu.y[loo_cosmo_idxs], emu.yerr[loo_cosmo_idxs]

In [ ]:
model = emu._emulator
model.compute(train_x, train_yerr)

In [ ]:
pred_y = model.predict(train_y, test_x, False, False, False)*emu._y_std + emu._y_mean


In [ ]:
np.mean(np.abs((pred_y-test_y)/test_y))
#np.mean(np.abs((pred_y-train_y)/train_y))

In [ ]:
        queue_skipper: True
        system: sherlock
        n_jobs: 400
        max_time: 6
resids = np.abs(emu.y*emu._y_std+emu._y_mean - ypred)

In [ ]:
np.mean(resids/(emu.y*emu._y_std+emu._y_mean))

In [ ]:
ypred.mean(), emu._y_mean

In [ ]:
test_gof = emu.goodness_of_fit(test_file, statistic = 'log_frac')
print test_gof.mean()

In [ ]:
test_gof = emu.goodness_of_fit(test_file, statistic = 'frac')
print test_gof.mean()

In [ ]:
plt.hist(np.log10(test_gof));

In [ ]:
test_x

In [ ]:
(emu.x*emu._x_std) + emu._x_mean

In [ ]:
emu.get_param_names()

In [ ]:
test_x_white, test_y_white = (test_x - emu._x_mean)/(emu._x_std + 1e-5), (test_y - emu._y_mean)/(emu._y_std + 1e-5)

In [ ]:
model = emu._emulator

In [ ]:
pred_y_white = model.predict(emu.y, test_x_white, False, False, False)

In [ ]:
pred_y = pred_y_white*emu._y_std + emu._y_mean

In [ ]:
plt.plot(pred_y[:100], label = 'pred')
plt.plot(test_y[:100], label = 'truth')

plt.legend(loc = 'best')

In [ ]:
test_y.mean(), emu._y_mean, pred_y.mean()

In [ ]:
test_y.std(), emu._y_std, pred_y.std()

In [ ]:
plt.hist(pred_y_white, bins = np.linspace(-3, 3, 100), label = 'Pred')
plt.hist(test_y_white, bins = np.linspace(-3, 3, 100), label = 'Test', alpha = 0.4);
plt.legend(loc = 'best')